In [1]:
%cd C:\Users\douget\Documents\Ethan\DataAnalysis\arrowhead_analytics\next-gen-scrapy-master

C:\Users\douget\Documents\Ethan\DataAnalysis\arrowhead_analytics\next-gen-scrapy-master


In [21]:
import requests 
from bs4 import BeautifulSoup 
import re
import json
import urllib
import os
import urllib.request
import cv2
import json
import scipy.misc
import pandas as pd
from undistort_field import *
from pass_detection import *

In [3]:
"""
Author: Sarah Mallepalle (updated to work in Python 3 by Ethan Douglas)

For a single pass chart image in 'Pass_Charts', extract only the trapezoidal image of the field, 
undistort the field by turning the trapezoid into a rectangle, remove the sideline labels, 
and save the new image to the folder 'Cleaned_Pass_Charts'.
"""

import cv2
import os
from skimage import data, color, img_as_ubyte, io
from skimage import feature
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter
import math
from PIL import Image
from PIL import ImageDraw
import numpy as np



def get_top(image):
	"""
	Function to get the pixel location of the top of the trapezoidal field.
	
	Input:
		image: image from the folder 'Pass_Charts'
	Return:
		top_left: location of the top left of the trapezoidal field
	"""
	frame = cv2.imread(image, 0)
	ret, img = cv2.threshold(frame, 40, 255, cv2.THRESH_BINARY_INV)

	points = np.fliplr(np.argwhere(img==0))

	mid = 600
	top = None
	top_left = None
	top_right = None

	for p in points:
		if (p[0] == mid):
			if (top == None): 
				top = p[1]
				break

	for p in points:
		if (p[1] == top):
			if (top_left == None):
				top_left = p[0]
				break

	return top_left

def make_grey_border(image):
	"""
	Function to add a grey border to the left and right of the trapezoidal field, 
	in order to undistort the trapezoid.
	
	Input:
		image: image from the folder 'Pass_Charts'
	Return:
		top_left: location of the top left of the trapezoidal field
		bordersize: width of the border in pixels added to the image
		border: numpy.ndarray representation of input image with border
		
	"""
	top_left = get_top(image)
	im = cv2.imread(image)
	row, col = im.shape[:2]
	
	grey_color = [108,96,86]

	start = 0
	for i in range(row):
		if (im[i,0,2] > 70): 
			start = i 
			break

	bordersize = int(math.ceil(float(top_left*row)/float(start)) - top_left)

	border = cv2.copyMakeBorder(im, top=0, bottom=0, 
		left=bordersize, right=bordersize, 
		borderType= cv2.BORDER_CONSTANT, value=grey_color)

	return top_left, bordersize, border

def undistort_field(image):
	"""
	Function to undistort the field by turning the trapezoid field image into a rectangle.
	
	Input:
		image: image from the folder 'Pass_Charts'
	Return:
		im_out: undistorted image of the field turned into a rectangle
	"""

	tl, bs, border_image = make_grey_border(image)

	image = cv2.imread(image)

	i_row, i_col = image.shape[:2]
	b_row, b_col = border_image.shape[:2]

	if b_col > 1398: return None
	pts_src = np.array([[0, i_row], [tl+bs, 0], [i_col-tl+bs, 0],[b_col, i_row]])
	pts_dst = np.array([[0, b_row],[0, 0],[b_col, 0],[b_col, b_row]])

	h, status = cv2.findHomography(pts_src, pts_dst)

	im_out = cv2.warpPerspective(border_image, h, (b_col, b_row)) 

	return im_out

def clean_field_70(image):
	"""
	Function that removes the sidelines of an undistorted pass chart field image,
	if the field image shows +70 yards from the line of scrimmage.
	
	Input:
		image: undistorted image, im_out
	Return:
		img: cleaned undistorted image, without sidelines
	"""
	img = Image.open(image)
	grey_color = (86,96,108)
	LOS1 = ((18,587), (86, 601))
	LOS2 = ((1308, 601), (1374,587))

	x0 = 33
	x1 = 0
	x2 = 1362
	x3 = max(img.size[0], 1394)

	l10 = ((x0, 520), (x1, 503))
	r10 = ((x2, 520), (x3, 505))

	l20 = ((x0, 440), (x1, 421))
	r20 = ((x2, 440), (x3, 423))

	l30 = ((x0, 360), (x1, 338))
	r30 = ((x2, 360), (x3, 340))

	l40 = ((x0, 279), (x1, 255))
	r40 = ((x2, 279), (x3, 257))

	l50 = ((x0, 200), (x1, 173))
	r50 = ((x2, 200), (x3, 175))

	l60 = ((x0, 118), (x1, 90))
	r60 = ((x2, 118), (x3, 92))

	l70 = ((x0, 38), (x1, 8))
	r70 = ((x2, 38), (x3, 10))

	sidelines = [LOS1, LOS2, l10, r10, l20, r20, l30, r30,
	            l40, r40, l50, r50, l60, r60, l70, r70
	]



	draw = ImageDraw.Draw(img)

	for number in sidelines: 
		draw.rectangle(number, fill=grey_color)

	return img

def clean_field_50(image):
	"""
	Function that removes the sidelines of an undistorted pass chart field image,
	if the field image shows +50 yards from the line of scrimmage.
	
	Input:
		image: undistorted image, im_out
	Return:
		img: cleaned undistorted image, without sidelines
	"""
	img = Image.open(image)
	grey_color = (86,96,108)
	LOS1 = ((20, 562), (84, 578))
	LOS2 = ((1340, 562), (1278, 578))

	x0 = 33
	x1 = 0
	x2 = 1331
	x3 = max(img.size[0], 1362)

	l10 = ((x0, 476), (x1, 455))
	r10 = ((x2, 476), (x3, 457))

	l20 = ((x0, 373), (x1, 349))
	r20 = ((x2, 373), (x3, 351))

	l30 = ((x0, 269), (x1, 243))
	r30 = ((x2, 269), (x3, 245))

	l40 = ((x0, 165), (x1, 137))
	r40 = ((x2, 165), (x3, 139))

	l50 = ((x0, 60), (x1, 30))
	r50 = ((x2, 60), (x3, 32))

	sidelines = [LOS1, LOS2, l10, r10, l20, r20, 
				l30, r30, l40, r40, l50, r50
	]



	draw = ImageDraw.Draw(img)

	for number in sidelines: 
		draw.rectangle(number, fill=grey_color)

	return img

def clean_field(image):
	"""
	Wrapper function for clean_field_50 and clean_field_70. 
	"""
	u_img = undistort_field(image)
	if u_img is None: 
		return None
	row, col = u_img.shape[:2]

	cv2.imwrite(image, u_img)

	if col > 1370:
		return clean_field_70(image)

	else:
		return clean_field_50(image)





In [5]:
teams = ["arizona-cardinals",
	"atlanta-falcons",
	"baltimore-ravens",
	"buffalo-bills",
	"carolina-panthers",
	"chicago-bears",
	"cincinnati-bengals",
	"cleveland-browns",
	"dallas-cowboys",
	"denver-broncos",
	"detroit-lions",
	"green-bay-packers",
	"houston-texans",
	"indianapolis-colts",
	"jacksonville-jaguars",
	"kansas-city-chiefs",
	"los-angeles-chargers",
	"los-angeles-rams",
	"miami-dolphins",
	"minnesota-vikings",
	"new-england-patriots",
	"new-orleans-saints",
	"new-york-giants",
	"new-york-jets",
	"oakland-raiders",
	"philadelphia-eagles",
	"pittsburgh-steelers",
	"san-francisco-49ers",
	"seattle-seahawks",
	"tampabay-buccaneers",
	"tennessee-titans",
	"washington-redskins"
]

seasons = ["2019"]

weeks = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13",
"14", "15", "16", "17", "wild-card", "divisional", "conference", "super-bowl"]

pattern = re.compile("charts")

print("Scraping images and html data...")
for team in teams:
	for season in seasons:
		print(team, "\t", season)
		for week in weeks:
			URL = "https://nextgenstats.nfl.com/charts/list/pass/" + team + "/" + season + "/" + week
			r = requests.get(URL)

			soup = BeautifulSoup(r.content, "html.parser")

			script = soup.find_all("script", text=pattern)

			contains_charts = json.loads(str(script[0])[33:-131])

			if (len(contains_charts["charts"]["charts"]) != 0):
				for chart in contains_charts["charts"]["charts"]['charts']:

					name = chart["lastName"] + "_" + chart["firstName"] + "_" + chart["position"]
					chart["team"] = team

					folder = str("Pass_Charts" + os.sep + team + os.sep + season + os.sep + week + os.sep)
					img_folder = folder + "images" + os.sep
					data_folder = folder + "data" + os.sep

					if not os.path.exists(img_folder):
						os.makedirs(img_folder)
					if not os.path.exists(data_folder):
						os.makedirs(data_folder)

					img_file = img_folder + name + ".jpeg"
					url = "https:" + chart["extraLargeImg"]
					urllib.request.urlretrieve(url, img_file)

					data_file = data_folder + name + ".txt"
					with open(data_file, 'w') as datafile: 
						json.dump(chart, datafile)

print("Done.")


Scraping images and html data...
arizona-cardinals 	 2019
atlanta-falcons 	 2019
baltimore-ravens 	 2019
buffalo-bills 	 2019
carolina-panthers 	 2019
chicago-bears 	 2019
cincinnati-bengals 	 2019
cleveland-browns 	 2019
dallas-cowboys 	 2019
denver-broncos 	 2019
detroit-lions 	 2019
green-bay-packers 	 2019
houston-texans 	 2019
indianapolis-colts 	 2019
jacksonville-jaguars 	 2019
kansas-city-chiefs 	 2019
los-angeles-chargers 	 2019
los-angeles-rams 	 2019
miami-dolphins 	 2019
minnesota-vikings 	 2019
new-england-patriots 	 2019
new-orleans-saints 	 2019
new-york-giants 	 2019
new-york-jets 	 2019
oakland-raiders 	 2019
philadelphia-eagles 	 2019
pittsburgh-steelers 	 2019
san-francisco-49ers 	 2019
seattle-seahawks 	 2019
tampabay-buccaneers 	 2019
tennessee-titans 	 2019
washington-redskins 	 2019
Done.


In [8]:
def new_image(image):

	img_name = image.split(os.sep)[-1].split(".")[0]
	img = cv2.imread(image)

	if (img.shape[0:2] == (1200, 1200)):
		crop_img = img[0:680, 0:1200]
		temp_name = img_name + "_temp.jpg"
		cv2.imwrite(temp_name, crop_img)
	else:
		print("Image must be of size (1200, 1200)")
		return

	clean_img = clean_field(temp_name)
	write_path = clean_path + os.sep + os.sep.join(image.split(os.sep)[1:-1]) 
	if not os.path.exists(write_path): os.makedirs(write_path)

	if (clean_img != None):
		write_name = write_path + os.sep + img_name + ".jpeg"
		scipy.misc.imsave(write_name, clean_img)
	os.remove(temp_name)

def new_data(folder, image): 
	data_path = os.sep.join(folder.split(os.sep)[:-1]) + os.sep + "data" 
	data_file = data_path + os.sep + image.split(".")[0] + ".txt"
	new_data_path = clean_path + os.sep + os.sep.join(folder.split(os.sep)[1:-1]) + os.sep + "data" 
	new_data_file = new_data_path + os.sep + image.split(".")[0] + ".txt"


	if not os.path.exists(new_data_path): 
		os.makedirs(new_data_path)

	with open(data_file) as _file: 
		old_data = json.load(_file)
		new_data = {key: old_data[key] for key in keys}
	with open(new_data_file, "w") as _file:
		json.dump(new_data, _file)



if __name__ == '__main__':
	keys = ['completions', 'passingYards', 'touchdowns', 'playerNameSlug', 
		'teamId', 'interceptions',  'playerName', 'season', 'position',
		'type', 'week', 'gameId', 'esbId', 'firstName', 'lastName', 
		'attempts', 'team'
	]

	clean_path = "Cleaned_Pass_Charts"

	if not os.path.exists(clean_path): os.makedirs(clean_path)

	pass_chart_folders = [folder[0] for folder in os.walk("Pass_Charts")]
	image_folders = [folder for folder in pass_chart_folders if folder.split(os.sep)[-1] == "images"]
	print("Cleaning images and data...")
	for folder in image_folders:
		#print folder 
		images = os.listdir(folder)
		for image in images:
			if not image.startswith("."): 
				new_image(os.path.join(folder, image))
				new_data(folder, image)
	print("Done.")


Cleaning images and data...


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Done.


In [20]:
def get_pass_data(data_file): 
	"""
	Extract the number of complete passes, incomplete passes, touchdowns, and interceptions 
	from a pass chart. 
	"""
	with open(data_file) as _file: 
		data = json.load(_file)
		_file.close()
	n_completions = data["completions"] - data["touchdowns"]
	n_touchdowns = data["touchdowns"]
	n_interceptions = data["interceptions"]
	n_incompletes = data["attempts"] -  n_completions -  n_interceptions - n_touchdowns

	return (n_completions, n_touchdowns, n_interceptions, n_incompletes)

def get_image(folder, data_file):
	"""
	If cleaned image exists, return the file path to the image, 
	otherwise return none.
	"""
	images_path = os.sep.join(folder.split(os.sep)[:-1]) + os.sep + "images" 
	image_file = images_path + os.sep + data_file.split(".")[0] + ".jpeg"
	if not os.path.exists(image_file):
		return None
	else:
		return image_file

def get_game_data(data_file):
	"""
	Extract player name, team, and game ID from the data corresponding to a pass chart image.
	"""	
	with open(data_file) as _file: 
		data = json.load(_file)
		_file.close()
	name = data["firstName"] + " " + data["lastName"]
	team = data["team"]
	game_id = data["gameId"]
	week = data_file.split(os.sep)[-3]
	return (name, team, game_id, week)

def write_pass_locations(image, data, passes):
	"""
	Write player, team, and game information, and locations of all passes to a .csv file.
	"""
	(n_com, n_td, n_int, n_inc) = get_pass_data(data)
	(name, team, game_id, week) = get_game_data(data)
	n_total = n_com + n_td + n_int + n_inc

	pass_cols = ["pass_type", "x", "y"]
	game_cols = ["game_id", "team", "week", "name"]
	pass_df = pd.DataFrame(columns = pass_cols)

	if (image is None): 
		rows_com = pd.DataFrame([["COMPLETE", None, None]]*n_com, 
			columns = pass_cols)
		rows_td = pd.DataFrame([["TOUCHDOWN", None, None]]*n_td, 
			columns = pass_cols)
		rows_int = pd.DataFrame([["INTERCEPTION", None, None]]*n_int, 
			columns = pass_cols)
		rows_inc = pd.DataFrame([["INCOMPLETE", None, None]]*n_inc, 
			columns = pass_cols)
		pass_df = pass_df.append([rows_com, rows_td, rows_int, rows_inc])
		game_df = pd.DataFrame([[game_id, team, week, name]]*pass_df.shape[0], 
			columns = game_cols)
		df = pd.concat([game_df, pass_df.reset_index(drop=True)], axis=1)
		passes = passes.append(df)
		return passes

	if n_com != 0: 
		rows_com = completions(image, n_com)
		pass_df = pass_df.append(rows_com)

	if n_td != 0: 
		rows_td = touchdowns(image, n_td)
		pass_df = pass_df.append(rows_td)

	if n_int != 0: 
		rows_int = interceptions(image, n_int)
		pass_df = pass_df.append(rows_int)

	if n_inc != 0: 
		rows_inc = incompletions(image, n_inc)
		pass_df = pass_df.append(rows_inc)

	game_df = pd.DataFrame([[game_id, team, week, name]]*pass_df.shape[0], 
		columns = game_cols)

	df = pd.concat([game_df, pass_df.reset_index(drop=True)], axis=1)
	passes = passes.append(df)
	return passes


if __name__ == '__main__':

	clean_path = "Cleaned_Pass_Charts"
	passes = pd.DataFrame(columns = ["game_id", "team", "week", "name", "pass_type", "x", "y"])

	pass_chart_folders = [folder[0] for folder in os.walk(clean_path)]
	data_folders = [folder for folder in pass_chart_folders if folder.split(os.sep)[-1] == "data"]

	print("Extracting pass locations...")
	for folder in data_folders:
		data = os.listdir(folder)
		print(folder)
		for data_file in data:
			if not data_file.startswith("."): 
				image = get_image(folder, data_file)
				passes = write_pass_locations(image, os.path.join(folder, data_file), passes)
	passes.to_csv("pass_locations.csv", index=False, header=True)
	print("Done.")

Extracting pass locations...
Cleaned_Pass_Charts\arizona-cardinals\2019\1\data


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:274: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Cleaned_Pass_Charts\arizona-cardinals\2019\2\data
Cleaned_Pass_Charts\atlanta-falcons\2019\1\data


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:274: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Cleaned_Pass_Charts\atlanta-falcons\2019\2\data


...

Cleaned_Pass_Charts\tennessee-titans\2019\3\data
Cleaned_Pass_Charts\washington-redskins\2019\1\data


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:274: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Cleaned_Pass_Charts\washington-redskins\2019\2\data


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:274: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Done.
